In [1]:
#!/usr/bin/env python
# coding: utf-8
from re import match
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import os, sys
import random
import datetime
import tqdm
from copy import deepcopy

import matplotlib.pyplot as plt

In [2]:
os.chdir("D://academic//match2")


number = 1000

dis_cache = None

#给乘客和车加各上一个id字段，方便索引和debug
class Worker:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y
    
    def __repr__(self) -> str:
        return '({},{})'.format(self.x, self.y)

class Task:
    def __init__(self, id, x, y, price):
        self.id = id
        self.x = x
        self.y = y
        self.price = price

    def __repr__(self) -> str:
        return '({},{}),{}'.format(self.x, self.y, self.price)

class Pair:
    def __init__(self, worker, task):
        self.worker = worker
        self.task = task
def dist(worker, task, cache=False):
    #空间换时间，理论上basic那次O(N^2)的循环里面这里都算了一遍，后面退火不用再算了
    if cache:
        r = dis_cache[worker.id][task.id]
        if r != -1:
            return r
    r2 = (worker.x - task.x) ** 2 + (worker.y - task.y) ** 2
    r = pow(r2, 0.5)
    if cache:
        dis_cache[worker.id][task.id] = r
    return r

In [3]:
'''
判断配对后的两对是否是不稳定对
不稳定的判断条件是同时满足：
pair1中的task1，相对于worker1，更喜欢pair2中的worker2，判断依据是worker2比worker1更近
pair2中的worker2，相对于task1，更喜欢pair1中的task1，判断依据是task1的price比task2更高

反之亦然
'''
def isUnstablePair(pair1, pair2):
    #只判断价钱高的那个task
    if pair1.task.price > pair2.task.price:
        if dist(pair1.worker, pair1.task) > dist(pair2.worker, pair1.task) and pair1.task.price > pair2.task.price:
            return True
    else:    
        if dist(pair2.worker, pair2.task) > dist(pair1.worker, pair2.task) and pair2.task.price > pair1.task.price:
            return True
    return False

#获取集合中距离某元素最近的一个值
def NN(element, set):
    y = None
    d = 0
    for p in set:
        d1 = dist(p, element)
        if d == 0 or d1 < d:
            d = d1
            y = p
    return y, d

#距离<=threshold的出价最高的task
def highest(w, tasks, threshold=100):
    best_task = None
    price, d = 0, 0
    for t in tasks:
        d1 = dist(t, w)
        if d1 <= threshold:
            if t.price > price:
                best_task = t
                price = t.price
                d = d1
    if best_task == None: #指定距离内无task时，返回最近task
        return NN(w, tasks)
    return best_task, d

In [4]:
def basic(number, workers, tasks):
    workers_num = len(workers)
    tasks_num = len(tasks)

    match = []

    def price(t):
        return t.price

    tasks.sort(key = price, reverse = True)
    match = [None for i in range(number)]

    starttime = datetime.datetime.now()
    dis_sum = 0
    for t in tqdm.tqdm(tasks):
        w_nearest = None
        d = 0
        for w in workers:
            d1 = dist(w, t)
            if d == 0 or d1 < d:
                d = d1
                w_nearest = w
        match[t.id] = w_nearest
        workers.remove(w_nearest)
        dis_sum += d

    endtime = datetime.datetime.now()
    time_cost = endtime - starttime
    print (str(number) + "个人和车的模拟数据集，传统双层循环配对消耗总时间：" + str(time_cost) + "，总距离为：" + str(dis_sum))    
    return match, dis_sum, str(time_cost)

In [5]:
def count_unstable(match, tasks, workers):
    #原始match: task.id -> worker
    #调整match: task.id -> worker.id
    match = [w.id for w in match]
    w2t = [-1] * len(match)
    for t in range(len(tasks)):
        w2t[match[t]] = t
    count = 0
    for t in tqdm.tqdm(range(len(tasks))):
        #t和新的w配对判断是否比t和match[t]配对更满意
        for w in range(len(workers)):
            if (dist(tasks[t], workers[match[t]]) > dist(tasks[t], workers[w])) and (tasks[t].price > tasks[w2t[w]].price):
                count += 1
                break
    return count

In [6]:
def is_unstable(pair1, pair2):
    if dist(pair1.worker, pair1.task) > dist(pair2.worker, pair1.task) and pair1.task.price > pair2.task.price:
        return True
    if dist(pair2.worker, pair2.task) > dist(pair1.worker, pair2.task) and pair2.task.price > pair1.task.price:
        return True
    return False

In [7]:
betas = [0.32768,0.49152,0.65536,1.31072,2.62144]
# betas=[2,4,8,16,32]
def anneal(number, match, workers, tasks, dis_sum_start):
    starttime = datetime.datetime.now()
#     percent=0.9
    H2 = H1 = dis_sum_start
    step = 1000000
    for i in tqdm.tqdm(range(step)):
        beta = betas[i//200000]
        t1 = tasks[np.random.randint(0, number)]
        t2 = tasks[np.random.randint(0, number)] 
        w1 = match[t1.id]
        w2 = match[t2.id]

        #随机交换之后不再满足稳定，直接continue


        H2 = H1 - dist(w2, t2) + dist(w2, t1) + dist(w1, t2) - dist(w1, t1)
        if H1 - H2 > 0:
            if is_unstable(Pair(w1, t2), Pair(w2, t1)):
                if np.random.rand() < percent:
                    continue
            H1 = H2
            match[t1.id] = w2
            match[t2.id] = w1
        elif rand() < np.exp(-beta * (H2-H1)):
            H1 = H2
            match[t1.id] = w2
            match[t2.id] = w1
    
    endtime = datetime.datetime.now()
    time_cost = endtime - starttime
    print (str(number) + "个人和车的模拟数据集，使用" + str(step) + "步退火消耗总时间：" + str(time_cost) + "，退火后总距离为：" + str(H1) + "，优化比例为：" + "{:.2%}".format(H1 / dis_sum_start))
    return H1,str(time_cost)
def anneal2(number, match, tasks, chain, dist_sum_first):
    starttime = datetime.datetime.now()
    dis_sum_start = dist_sum_first
    n=len(chain)
    for m in range(len(chain) - 2):
        m1, m2, m3 = chain[m], chain[m + 1], chain[m + 2]
        t1, t2, t3 = tasks[m1], tasks[m2], tasks[m3]
        w1, w2, w3 = match[m1], match[m2], match[m3]
        l0 = dist(w1, t1) + dist(w2, t2) + dist(w3, t3)
        l12 = dist(w1, t2) + dist(w2, t1) + dist(w3, t3)
        l13 = dist(w1, t3) + dist(w2, t2) + dist(w3, t1)
        l23 = dist(w1, t1) + dist(w2, t3) + dist(w3, t2)
        l_min = min(l0, l12, l13, l23)
        
        if l_min == l0:
            continue
        elif l_min == l12:
            match[m1] = w2
            match[m2] = w1
            dist_sum_first = dist_sum_first - l0 + l12
            n-=1
        elif l_min == l13:
            match[m1] = w3
            match[m3] = w1
            dist_sum_first = dist_sum_first - l0 + l13
            n -= 1
        elif l_min == l23:
            match[m3] = w2
            match[m2] = w3
            dist_sum_first = dist_sum_first - l0 + l23
            n -= 1
    endtime = datetime.datetime.now()
    time_cost = endtime - starttime

    print(str(number) + "个人和车的模拟数据集，使用" + str(len(chain) - 2) + "步退火消耗总时间：" + str(time_cost) + "，退火后总距离为：" + str(
        dist_sum_first) + "，优化比例为：" + "{:.2%}".format(1-dist_sum_first / dis_sum_start),"，稳定比例为：" + "{:.2%}".format(n/len(chain)))
    return time_cost, dist_sum_first,round(1-dist_sum_first / dis_sum_start,2),round(n/len(chain),2)

In [8]:
def load_dataset(number):
    workers = []
    tasks = []
    idx = 0
    with open("worker_" + str(number) + ".txt", "r") as f:
        while True:
            try:
                line = f.readline().split()
                w = Worker(idx, float(line[0]), float(line[1]))
                workers.append(w)
                idx += 1
            except:
                break
    idx = 0
    with open("task_" + str(number) + ".txt", "r") as f:
        while True:
            try:
                line = f.readline().split()
                t = Task(idx, float(line[0]), float(line[1]), float(line[2]))
                tasks.append(t)
                idx += 1
            except:
                break
    return tasks, workers

In [9]:

number=1000
percent=0.95
tasks, workers = load_dataset(number)
    #zero-based，方便空间换时间索引
workers_stable = deepcopy(workers)
tasks_stable = deepcopy(tasks)
workers_sc = deepcopy(workers)
tasks_sc = deepcopy(tasks)
match_result_1, dist_sum_1, time_cost_1 = basic(number, workers, tasks)
# cnt = count_unstable(match_result_2, tasks_stable, workers_stable)
# print("退火前不稳定个数是：" + str(cnt))
anneal(number, match_result_1, workers_stable, tasks_stable, dist_sum_1)
#退火后再计算下不稳定个数
cnt = count_unstable(match_result_1, tasks_stable, workers_stable)
print("退火后不稳定个数是：" + str(cnt))

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 3394.93it/s]


1000个人和车的模拟数据集，传统双层循环配对消耗总时间：0:00:00.310180，总距离为：27639.308844782834


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:10<00:00, 94459.39it/s]


1000个人和车的模拟数据集，使用1000000步退火消耗总时间：0:00:10.597360，退火后总距离为：20744.500416180577，优化比例为：75.05%


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1183.52it/s]

退火后不稳定个数是：458


In [10]:
print(np.random.rand())

0.8646519639486414


In [11]:
d=0
for i in tasks:
    d+=dist(i,match_result_2[i.id])
print(d)

NameError: name 'match_result_2' is not defined

In [ ]:
m=chain[0]


In [15]:
basic_dists = []

anneal_dists = []
basic_times = []

anneal_times = []
anneal_shorten=[]
anneal_broken=[]
cnt_lis=[]
nums = [500,1000 ,1500 ,2000,2500,3000]
#,8000,9000,10000,11000,12000,13000,14000,15000]
for number in nums:
    percent=0.99
    global dis_cache
    dis_cache =[[-1 for i in range(number)] for j in range(number)]
    tasks, workers = load_dataset(number)
    #zero-based，方便空间换时间索引
    workers_stable = deepcopy(workers)
    tasks_stable = deepcopy(tasks)
    workers_sc = deepcopy(workers)
    tasks_sc = deepcopy(tasks)
    match_result_1, dist_sum_1, time_cost_1 = basic(number, workers, tasks)
    basic_dists.append(dist_sum_1)
    basic_times.append(time_cost_1[5:])
    #基于chain出来的数据进行退火
    dist_sum_2,time_cost_2=anneal(number, match_result_1, workers_stable, tasks_stable, dist_sum_1)
    anneal_dists.append(dist_sum_2)
    anneal_times.append(time_cost_2[5:])
    #退火后再计算下不稳定个数
    cnt = count_unstable(match_result_1, tasks_stable, workers_stable)
    cnt_lis.append(cnt)
    print("退火后不稳定个数是：" + str(cnt))
    
#     anneal_times.append(time_cost_an.seconds)
#     anneal_dists.append(dist_sum_an)
#     anneal_shorten.append(shorten)
#     anneal_broken.append(1-stable)
# import numpy as np
# plt.figure()
# plt.plot(nums,basic_times)
# plt.plot(nums,stable_chain_times)
# plt.plot(nums,np.array(anneal_times)+np.array(stable_chain_times))
# plt.legend(['basic','stable-chain','anneal'])
# plt.title("basic & stable-chain time comparation")
# plt.savefig("result1.png")
# plt.figure()
# plt.plot(nums,basic_dists)
# plt.plot(nums,stable_chain_dists)
# plt.plot(nums,anneal_dists)
# plt.legend(['basic','stable-chain','anneal'])
# plt.title("basic & stable-chain distance comparation")
# plt.savefig("result2.png")
# plt.figure()
# plt.plot(nums,anneal_shorten)
# plt.plot(nums,anneal_broken)
# plt.legend(['shorten','broken'])
# plt.title("anneal comparation")
# plt.savefig("result3.png")

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 6270.79it/s]


500个人和车的模拟数据集，传统双层循环配对消耗总时间：0:00:00.081642，总距离为：14908.450051136992


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:10<00:00, 99445.78it/s]


500个人和车的模拟数据集，使用1000000步退火消耗总时间：0:00:10.058793，退火后总距离为：11703.77619445497，优化比例为：78.50%


100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 2006.81it/s]


...

3000个人和车的模拟数据集，传统双层循环配对消耗总时间：0:00:02.849197，总距离为：55997.68279675415


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:10<00:00, 96032.88it/s]


3000个人和车的模拟数据集，使用1000000步退火消耗总时间：0:00:10.421695，退火后总距离为：51260.99274604929，优化比例为：91.54%


100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:09<00:00, 331.09it/s]

退火后不稳定个数是：1123


In [16]:
import pandas as pd
import numpy as np
df=pd.DataFrame({"basic_dists":basic_dists,"basic_times":basic_times,"anneal_dists":anneal_dists,"anneal_times":anneal_times,"cnt":cnt_lis})
df.to_excel("data_0.99.xlsx")

In [ ]:
print(anneal_times)

In [ ]:
an=[float(i) for i in anneal_times]
ba=[float(i) for i in basic_times]
import numpy as np
import matplotlib.pyplot as plt
plt.figure()
y_tick = np.linspace(0,15,11)
plt.yticks(y_tick)
# plt.set_yticks(1.5*i for i in range(10))
plt.plot(nums,basic_times,'-o')
# plt.plot(nums,stable_chain_times)
plt.plot(nums,np.array(an)+np.array(ba),'-^')


plt.legend(['basic','anneal'])
plt.title("basic & anneal time comparation")
plt.savefig("time_"+str(percent)+".png")
plt.figure()
plt.plot(nums,basic_dists,'-o')
# plt.plot(nums,stable_chain_dists)
plt.plot(nums,anneal_dists,'-^')
plt.legend(['basic','anneal'])
plt.title("basic & anneal distance comparation")
plt.savefig("distance_"+str(percent)+".png")
# plt.figure()
# plt.plot(nums,anneal_shorten)
# plt.plot(nums,anneal_broken)
# plt.legend(['shorten','broken'])
# plt.title("anneal comparation")
# plt.savefig("result3.png")

In [ ]:
y_tick = np.linspace(0,15,11)
plt.yticks(y_tick)
plt.plot(nums,np.array(an)+np.array(ba),'-^')
plt.show()

In [ ]:
an=[float(i) for i in anneal_times]
ba=[float(i) for i in basic_times]
import numpy as np
import matplotlib.pyplot as plt
plt.figure()
y_tick = np.linspace(0,15,11)
plt.yticks(y_tick)
# plt.set_yticks(1.5*i for i in range(10))
plt.plot(nums,basic_times,'-o')
# plt.plot(nums,stable_chain_times)
plt.plot(nums,np.array(an)+np.array(ba),'-^')


plt.legend(['basic','anneal'])
plt.title("basic & anneal time comparation")
plt.savefig("time_"+str(percent)+".png")
plt.figure()